In [1]:
import sys
BASE_DIR = "/home/dzigen/Desktop/Projects/rag_project"
sys.path.insert(0, BASE_DIR)

import pandas as pd
import ast
import random
import json
import numpy as np
from tqdm import tqdm
random.seed(42)

from src.llm_agent.agent_connector import AgentConnectorConfig, AgentConnector
from src.utils import ReaderMetrics

BENCHMARK_PATH = "/home/dzigen/Desktop/Projects/rag_project/data/mtssquad/tables/v3/benchmark.csv"
CHUNKED_DOCS_PATH = "/home/dzigen/Desktop/Projects/rag_project/data/mtssquad/tables/v3/chunked_docs.csv"

In [2]:
PARAMS = {
    'version': 1,
    'num_samples': 500,
    'num_contexts': 5,
    'system_prompt': "Ты AI-асситент, который помогает решать пользовательские вопросы.",
    "item_format": "- [{score}] {document}",
    "user_prompt": 'Ответь на вопрос, используя доступную информацию из текстов в списке ниже. Каждому тексту в начале в квадратных скобках поставлена в соответствие вещественная оценка его релевантности по отношению к вопросу: в диапозоне от 0.0 (текст не подходит для генерации ответа на его основе) до 1.0 (текст подходит для генерации ответа на его основе). Используй эту информацию. Выбирай тексты c достаточно высокими оценками релевантности. Если на основании указанных оценок в списке нет достаточно релевантных текстов для генерации ответов на их основе, то сгенерируй следующий ответ: "У меня нет ответа на ваш вопрос.". Ответы генерируй только на русском языке. Не дублируй вопрос в ответе. Сгенерируй только ответ на указанный вопрос. Не генерируй ничего лишнего.',
    "prompt_format": "{user_p}\n\nДоступная информация:\n{cnt_list}\n\nВопрос:\n{q}\n\nОтвет:\n",
    'scores': {'rel': 1.0, 'unrel': 0.0},
    'gen_strat': {'max_new_tokens': 1024},
    'stub_answer': "У меня нет ответа на ваш вопрос."
}

PARAMS_SAVE_NAME = 'hyperp.json'
GEN_ANSW_SAVE_NAME = 'generation_info.json'
SCORES_SAVE_NAME = 'scores.json'

### Подключение к агенту

In [ ]:
agent = AgentConnector.open(AgentConnectorConfig())

In [ ]:
agent.generate("Сколько будет 2 + 2?")

### Формируем список контекстов для каждого запроса со скорами

In [3]:
chunks_df = pd.read_csv(CHUNKED_DOCS_PATH, sep=';')

benchmark_df = pd.read_csv(BENCHMARK_PATH, sep=';')
benchmark_df['chunk_ids'] = benchmark_df['chunk_ids'].apply(lambda v: ast.literal_eval(v)) 

In [4]:
CONTEXTS_LIST_IDS = []
for i in tqdm(range(PARAMS['num_samples'])):
    rel_id = benchmark_df['chunk_ids'][i][0]
    cur_list_ids = []

    while len(cur_list_ids) != PARAMS['num_contexts']:
        unrel_context_id = random.randint(0, chunks_df.shape[0]-1)

        if chunks_df['chunk_id'][unrel_context_id] == rel_id:
            continue        

        prep_cntx = (PARAMS['scores']['unrel'], chunks_df['chunk_id'][unrel_context_id])
        if prep_cntx not in cur_list_ids:
            cur_list_ids.append(prep_cntx)

    CONTEXTS_LIST_IDS.append(cur_list_ids)

100%|██████████| 500/500 [00:00<00:00, 19437.33it/s]


In [5]:
CONTEXTS_LIST_IDS[0]

[(0.0, 'd764509cd8bbeca063ad039f783937fd'),
 (0.0, '3296e4d68127ee5f3604f78d11642396'),
 (0.0, 'fce1f4f018a5a48dee1a0d7849189375'),
 (0.0, '50b5aa882912e4987405ed1551595f4f'),
 (0.0, 'b9a708c0396d19d2a3621bffb4744f37')]

### Готовим промпт

In [6]:
USER_PROMPTS = []
for i in tqdm(range(len(CONTEXTS_LIST_IDS))):
    documents_list = '\n'.join(list(map(lambda v: PARAMS['item_format'].format(
        score=v[0], document=chunks_df[chunks_df['chunk_id'] == v[1]]['chunks'].to_list()[0]), CONTEXTS_LIST_IDS[i])))

    USER_PROMPTS.append( PARAMS['prompt_format'].format(user_p=PARAMS['user_prompt'], cnt_list=documents_list, q=benchmark_df['question'][i]))

100%|██████████| 500/500 [00:01<00:00, 262.54it/s]


In [7]:
print(USER_PROMPTS[0])

Ответь на вопрос, используя доступную информацию из текстов в списке ниже. Каждому тексту в начале в квадратных скобках поставлена в соответствие вещественная оценка его релевантности по отношению к вопросу: в диапозоне от 0.0 (текст не подходит для генерации ответа на его основе) до 1.0 (текст подходит для генерации ответа на его основе). Используй эту информацию. Выбирай тексты c достаточно высокими оценками релевантности. Если на основании указанных оценок в списке нет достаточно релевантных текстов для генерации ответов на их основе, то сгенерируй следующий ответ: "У меня нет ответа на ваш вопрос.". Ответы генерируй только на русском языке. Не дублируй вопрос в ответе. Сгенерируй только ответ на указанный вопрос. Не генерируй ничего лишнего.

Доступная информация:
- [0.0] В рамках прямого финансирования: банк предоставляет компаниям малого и среднего бизнеса кредитно-гарантийную поддержку, в том числе по Программе стимулирования кредитования субъектов МСП ( Программа 6,5 ). Для сре

In [8]:
with open(f"./logs/v{PARAMS['version']}/user_prompts.json", 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(USER_PROMPTS, ensure_ascii=False, indent=1))

# сохраняем конфигурацию эксперимента
with open(f"./logs/v{PARAMS['version']}/{PARAMS_SAVE_NAME}", 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(PARAMS, ensure_ascii=False, indent=1))

### Генерируем ответы на вопросы

In [ ]:
generate_answers = []
display_iter = 2
for i in tqdm(range(len(USER_PROMPTS))):
    pred_answer = agent.generate(user_prompt=USER_PROMPTS[i], system_prompt=PARAMS['system_prompt'], gen_strategy=PARAMS['gen_strat'])
    generate_answers.append(pred_answer)

    if i % display_iter == 0:
        print(f"\n[{i}]: \nGEN: {pred_answer}\nGOLD: {benchmark_df['answer'][i]}")

### Оцениваем качество

In [4]:
with open('./logs/v1/generated_output.json','r', encoding='utf8') as fd:
    predicted_answers = list(map(lambda v: v[0], json.loads(fd.read())))

In [5]:
metrics = ReaderMetrics(base_dir="/home/dzigen/Desktop/Projects/rag_project", model_path='ru_electra_medium')

Loading Meteor...
Loading ExactMatch


In [6]:
target_scores = {
    'BLEU2': [], 'BLEU1': [],
    'ExactMatch': [],'METEOR': [],
    'BertScore': [],
    'Levenshtain': [],
    'ROUGEL': []}

stub_scores = {
    'BLEU2': [], 'BLEU1': [],
    'ExactMatch': [],'METEOR': [],
    'BertScore': [],
    'Levenshtain': [],
    'ROUGEL': []}

show_step = 10

process = tqdm(range(PARAMS['num_samples']))
target_answers =  benchmark_df['answer'].to_list()[:PARAMS['num_samples']]
tmp_stub_pred_answers = []
for i in process:
    
    predicted_answer = predicted_answers[i]
    target_answer = target_answers[i]

    target_scores['BLEU1'] += metrics.bleu1([predicted_answer], [target_answer])
    target_scores['BLEU2'] += metrics.bleu2([predicted_answer], [target_answer])
    target_scores['ExactMatch'] += metrics.exact_match([predicted_answer], [target_answer])
    target_scores['METEOR'] += metrics.meteor([predicted_answer], [target_answer])
    target_scores['Levenshtain'] += metrics.levenshtain_score([predicted_answer], [target_answer])
    target_scores['ROUGEL'] += metrics.rougel([predicted_answer], [target_answer])


    stub_pred_answer = predicted_answer[-len(PARAMS['stub_answer']):]
    tmp_stub_pred_answers.append(stub_pred_answer)

    stub_scores['BLEU1'] += metrics.bleu1([stub_pred_answer], [PARAMS['stub_answer']])
    stub_scores['BLEU2'] += metrics.bleu2([stub_pred_answer], [PARAMS['stub_answer']])
    stub_scores['ExactMatch'] += metrics.exact_match([stub_pred_answer], [PARAMS['stub_answer']])
    stub_scores['METEOR'] += metrics.meteor([stub_pred_answer], [PARAMS['stub_answer']])
    stub_scores['Levenshtain'] += metrics.levenshtain_score([stub_pred_answer], [PARAMS['stub_answer']])
    target_scores['ROUGEL'] += metrics.rougel([stub_pred_answer], [PARAMS['stub_answer']])
            
    if i % show_step == 0:
        process.set_postfix({m_name: np.mean(score) for m_name, score in stub_scores.items()})

target_scores = {m_name: round(float(np.mean(score)), 5) for m_name, score in target_scores.items()}
target_scores['BertScore'] = metrics.bertscore(predicted_answers, target_answers)

stub_scores = {m_name: round(float(np.mean(score)), 5) for m_name, score in stub_scores.items()}
stub_scores['BertScore'] = metrics.bertscore(tmp_stub_pred_answers, [PARAMS['stub_answer']]*len(tmp_stub_pred_answers))
stub_scores['elapsed_time_sec'] = round(float(process.format_dict["elapsed"]), 3)

  0%|          | 0/500 [00:00<?, ?it/s]/home/dzigen/miniconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dzigen/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 500/500 [00:39<00:00, 12.60it/s, BLEU2=1, BLEU1=1, ExactMatch=1, METEOR=0.999, BertScore=nan, Levenshtain=0, ROUGEL=nan]
/home/dzigen/miniconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
with open(f"./logs/v{PARAMS['version']}/{SCORES_SAVE_NAME}", 'w', encoding='utf-8') as fp:
    fp.write(json.dumps({'target_answers': target_scores, 'stub_answers': stub_scores}, ensure_ascii=False, indent=1))